<a href="https://colab.research.google.com/github/thesteve0/impatient-computer-vision/blob/main/7_visual_language_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Let's do the housekeeping

# Visual Language Models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install fiftyone==1.4.1 torch torchvision umap-learn

import fiftyone as fo

name = "our-photos"
dir = "/content/drive/MyDrive/impatient-cv/flickr-labeled"

dataset = fo.Dataset.from_dir(
    dataset_dir=dir,
    dataset_type=fo.types.FiftyOneDataset,
    name=name
)


## Running models

There are all sorts of interesting tasks you can do with these multi-modal models, for example, [this code](https://github.com/thesteve0/photo-explorer/blob/main/4_word_cloud.py) generates a word cloud from ideas in the dataset's images. Today we are going to use these models to help us generate annotations in a dataset without annotations.